In [ ]:

# ========== Configuration ==========
import os
ROOT_DIR = "/Users/chunghyunhan/Projects/agentics/Decision_runs_result_saving"  
OUT_CSV  = os.path.join(ROOT_DIR, "decision_runs_consolidated.csv")
OUT_JSONL= os.path.join(ROOT_DIR, "decision_runs_consolidated.jsonl")

print("ROOT_DIR:", ROOT_DIR)


ROOT_DIR: /Users/chunghyunhan/Projects/agentics/Decision_runs_result_saving


In [2]:

# ========== Helpers ==========
import json, os, glob, pandas as pd
from typing import Any, Dict, List

def safe_get(d: Dict[str, Any], path: List[Any], default=None):
    cur = d
    for p in path:
        if isinstance(cur, dict) and p in cur:
            cur = cur[p]
        else:
            return default
    return cur

def flatten_evidence_quotes(evidence_list):
    if not isinstance(evidence_list, list):
        return ""
    return " | ".join([ev.get("quote") for ev in evidence_list if isinstance(ev, dict) and ev.get("quote")])

def collect_similar_ids(sim_list):
    if not isinstance(sim_list, list):
        return ""
    ids = []
    for x in sim_list:
        pid = (x.get("proposal_id") or (x.get("cleaned") or {}).get("proposal_id"))
        if pid:
            ids.append(pid)
    return ",".join(ids)

def build_similar_cleaned_columns(sim_list: List[Dict[str, Any]]) -> Dict[str, Any]:
    """
    Flatten ALL 'cleaned' fields from similar_proposals_data with numbered prefixes,
    but DROP 'winning_option_index' as requested.
    """
    out = {}
    if not isinstance(sim_list, list):
        return out
    for i, item in enumerate(sim_list, start=1):
        cleaned = item.get("cleaned") or {}
        for k, v in cleaned.items():
            if k == "winning_option_index":  # drop
                continue
            out[f"similar_proposal_{i}_{k}"] = v
    return out

def build_similar_raw_impacts(sim_list: List[Dict[str, Any]]) -> Dict[str, Any]:
    """
    Split RAW impacts per similar index (no list columns):
      similar_proposal_{i}_price_impact_pct_raw
      similar_proposal_{i}_tvl_impact_pct_raw
    """
    out = {}
    if not isinstance(sim_list, list):
        return out
    for i, item in enumerate(sim_list, start=1):
        raw = item.get("raw") or {}
        tvl_pct   = (raw.get("tvl_impact") or {}).get("abnormal_change_pct")
        price_pct = (raw.get("price_impact") or {}).get("abnormal_change_pct")
        if price_pct is not None:
            out[f"similar_proposal_{i}_price_impact_pct_raw"] = price_pct
        if tvl_pct is not None:
            out[f"similar_proposal_{i}_tvl_impact_pct_raw"]   = tvl_pct
    return out

def extract_record(d: Dict[str, Any]) -> Dict[str, Any]:
    snapshot_url = d.get("snapshot_url") or safe_get(d, ["decision","snapshot_url"], "")
    tmc = d.get("timeline_metrics_current") or {}
    decision = d.get("decision", {})
    simdata  = d.get("similar_proposals_data", [])

    avr = decision.get("actual_vote_result") or {}
    sim_props = decision.get("similar_proposals") or []

    # Base fields (we'll reorder later to your exact sequence)
    row = {
        "snapshot_url": snapshot_url,
        "summary": decision.get("summary"),
        "total_votes": tmc.get("total_votes"),
        "vp_by_quartile": json.dumps(tmc.get("vp_by_quartile")) if tmc.get("vp_by_quartile") is not None else None,

        "available_choices": ", ".join(decision.get("available_choices") or []) if isinstance(decision.get("available_choices"), list) else decision.get("available_choices"),

        "actual_outcome": d.get("actual_outcome"),
        "actual_vote_margin_pct": avr.get("margin_pct"),
        "actual_vote_scores_total": avr.get("scores_total"),
        "agentic_ai_choice": d.get("agentic_ai_choice"),
        "ai_final_conclusion": decision.get("ai_final_conclusion"),
        "ai_final_reason": decision.get("ai_final_reason"),
        "decision_stance": decision.get("decision_stance"),
        "evidence_quotes": flatten_evidence_quotes(decision.get("evidence")),
        "forum_sentiment_summary": json.dumps(d.get("forum_sentiment_summary")) if d.get("forum_sentiment_summary") else None,
        "half_slope_diff": tmc.get("half_slope_diff"),
        "spike_follow_support_ratio": tmc.get("spike_follow_support_ratio"),
        "spike_index": tmc.get("spike_index"),
        "stairwise_ratio": tmc.get("stairwise_ratio"),

        "ex_post_price_impact_pct": decision.get("ex_post_price_impact_pct"),
        "ex_post_tvl_impact_pct": decision.get("ex_post_tvl_impact_pct"),

        "key_arguments_against": " | ".join(decision.get("key_arguments_against") or []),
        "key_arguments_for": " | ".join(decision.get("key_arguments_for") or []),
        "selected_choice_label": decision.get("selected_choice_label"),
        "match_result": d.get("match_result"),

        "similar_proposals": collect_similar_ids(sim_props),
    }

    # similar cleaned (without winning_option_index) + per-index RAW fields
    row.update(build_similar_cleaned_columns(simdata))
    row.update(build_similar_raw_impacts(simdata))
    return row


In [30]:
# Load existing to count NEW
if os.path.exists(OUT_CSV):
    existing_df = pd.read_csv(OUT_CSV)
    known_urls = set(existing_df.get("snapshot_url", pd.Series(dtype=str)).dropna().astype(str))
else:
    existing_df = pd.DataFrame()
    known_urls = set()

# Read all JSON and extract
rows = []
for fp in sorted(glob.glob(os.path.join(ROOT_DIR, "*.json"))):
    try:
        with open(fp, "r") as f:
            data = json.load(f)
    except Exception:
        continue
    rows.append(extract_record(data))

new_df = pd.DataFrame(rows)

# Count newly seen snapshot_urls
file_urls = set(new_df.get("snapshot_url", pd.Series(dtype=str)).dropna().astype(str))
new_urls = file_urls - known_urls
num_new = len(new_urls)

# ===== Union schema & upsert (prefer latest by snapshot_url), ALWAYS overwrite outputs =====
union_cols = sorted(set(existing_df.columns) | set(new_df.columns))
existing_df = existing_df.reindex(columns=union_cols, fill_value=pd.NA)
new_df      = new_df.reindex(columns=union_cols,      fill_value=pd.NA)

# 빈 DF 제외하고 concat → FutureWarning 방지
frames = [df for df in (existing_df, new_df) if not df.empty]
if frames:
    final_df = pd.concat(frames, ignore_index=True)
else:
    final_df = pd.DataFrame(columns=union_cols)

# 중복 제거 (latest 우선)
if "snapshot_url" in final_df.columns:
    final_df = final_df.drop_duplicates(subset=["snapshot_url"], keep="last")

# ===== (a) DROP unwanted columns =====
cols_to_drop = {
    "actual_vote_result_winner",
    "similar_raw_price_impact_pct_list",
    "similar_raw_tvl_impact_pct_list",
}
for c in list(final_df.columns):
    if c.startswith("similar_proposal_") and c.endswith("_winning_option_index"):
        cols_to_drop.add(c)

final_df = final_df.drop(columns=[c for c in cols_to_drop if c in final_df.columns], errors="ignore")

# ===== (b) REORDER columns to the exact guide =====
base_order = [
    "snapshot_url",
    "summary",
    "total_votes",
    "vp_by_quartile",

    "available_choices",

    "actual_outcome",
    "decision_stance",
    "actual_vote_margin_pct",
    # "actual_vote_result_winner" removed
    "actual_vote_scores_total",
    "agentic_ai_choice",
    "ai_final_conclusion",
    "ai_final_reason",
    
    "evidence_quotes",
    "forum_sentiment_summary",
    "half_slope_diff",
    "spike_follow_support_ratio",
    "spike_index",
    "stairwise_ratio",

    "ex_post_price_impact_pct",
    "ex_post_tvl_impact_pct",

    "key_arguments_against",
    "key_arguments_for",
    "selected_choice_label",
    "match_result",

    "similar_proposals",
]

def sim_cleaned_fields(i: int):
    return [
        f"similar_proposal_{i}_author",
        f"similar_proposal_{i}_change_stance",
        f"similar_proposal_{i}_end_utc",
        f"similar_proposal_{i}_margin_abs",
        f"similar_proposal_{i}_margin_pct",
        f"similar_proposal_{i}_proposal_id",
        f"similar_proposal_{i}_scores_total",
        f"similar_proposal_{i}_similarity_score",
        f"similar_proposal_{i}_title",
        f"similar_proposal_{i}_winning_option",
    ]

def sim_raw_fields(i: int):
    return [
        f"similar_proposal_{i}_price_impact_pct_raw",
        f"similar_proposal_{i}_tvl_impact_pct_raw",
    ]

pattern = re.compile(r"^similar_proposal_(\d+)_")
indices_found = set()
for c in final_df.columns:
    m = pattern.match(c)
    if m:
        indices_found.add(int(m.group(1)))
max_idx = min(max(indices_found) if indices_found else 0, 5)  # order up to 5

target_order = list(base_order)
for i in range(1, max_idx + 1):
    target_order += sim_cleaned_fields(i)
for i in range(1, max_idx + 1):
    target_order += sim_raw_fields(i)

# 실제 존재하는 컬럼만 순서 적용 + 나머지 뒤에 유지
existing_cols_in_order = [c for c in target_order if c in final_df.columns]
rest = [c for c in final_df.columns if c not in existing_cols_in_order]
final_df = final_df.reindex(columns=existing_cols_in_order + rest)

# ===== Save (overwrite) =====
final_df.to_csv(OUT_CSV, index=False)
# with open(OUT_JSONL, "w") as f:
#     for _, r in final_df.iterrows():
#         f.write(json.dumps({k:v for k,v in r.items() if pd.notna(v)}, ensure_ascii=False) + "\\n")

print(f"[NEW ADDED THIS RUN]: {num_new}")
print(f"[TOTAL ROWS NOW]    : {len(final_df)}")
print(f"CSV   -> {OUT_CSV}")

# Small preview
try:
    from caas_jupyter_tools import display_dataframe_to_user
    preview_cols = [c for c in final_df.columns if c.startswith("similar_proposal_")][:10]
    display_dataframe_to_user("Preview (first 5 rows)", final_df[["snapshot_url"] + preview_cols].head(5))
except Exception:
    display(final_df.head(5))


[NEW ADDED THIS RUN]: 0
[TOTAL ROWS NOW]    : 145
CSV   -> /Users/chunghyunhan/Projects/agentics/Decision_runs_result_saving/decision_runs_consolidated.csv


,snapshot_url,summary,total_votes,vp_by_quartile,available_choices,actual_outcome,decision_stance,actual_vote_margin_pct,actual_vote_scores_total,agentic_ai_choice,...,similar_proposal_1_price_impact_pct_raw,similar_proposal_1_tvl_impact_pct_raw,similar_proposal_2_price_impact_pct_raw,similar_proposal_2_tvl_impact_pct_raw,similar_proposal_3_price_impact_pct_raw,similar_proposal_3_tvl_impact_pct_raw,similar_proposal_4_price_impact_pct_raw,similar_proposal_4_tvl_impact_pct_raw,similar_proposal_5_price_impact_pct_raw,similar_proposal_5_tvl_impact_pct_raw
145,https://snapshot.org/#/aavedao.eth/proposal/0x...,The proposal to onboard MetaMask USD (mUSD) to...,111,"[[467037.7905335825, 0.0, 0.0], [335255.968332...","For, Against, Abstain",For,Change,0.999993,891902.299606,For,...,1.0231,3.6680,0.6381,-5.9885,0.5070,-0.7881,-1.2512,-7.9590,-6.4433,12.7784
146,https://snapshot.org/#/aavedao.eth/proposal/0x...,The proposal to automate the Slope2 parameter ...,122,"[[136969.2662189785, 0.03937986209741342, 0.0]...","YAE, NAY, Abstain",YAE,Change,0.999993,861399.335022,YAE,...,-1.7802,5.2431,7.8286,-1.7742,17.1318,23.6390,4.1309,6.4144,-12.0585,-15.1163
147,https://snapshot.org/#/aavedao.eth/proposal/0x...,Proposal to launch GHO on Ink and set ACI as e...,99,"[[771536.9889235826, 0.09615768208900824, 0.0]...","For, Against, Abstain",For,Change,1.000000,890826.139112,For,...,-3.9720,4.4550,1.0231,3.6680,1.0231,3.6680,-7.7784,-6.0961,4.6309,5.7356
148,https://snapshot.org/#/aavedao.eth/proposal/0x...,This proposal seeks to onboard tBTC to Aave v3...,98,"[[66784.87231949075, 0.0141, 0.0437], [771521....","YAE, NAY, Abstain",YAE,Change,0.999649,890911.270993,YAE,...,-16.6566,-4.0773,13.6563,0.8474,0.9369,2.7357,-15.1188,-4.5017,-6.6626,-14.1257
149,https://snapshot.org/#/aavedao.eth/proposal/0x...,"The proposal to add XAUt, a Tether gold-backed...",99,"[[66785.05378354878, 0.0, 0.0], [771521.632339...","YAE, NAY, Abstain",YAE,Change,0.999454,890932.203059,YAE,...,-6.5765,2.3709,5.9126,7.9371,-10.9111,-2.2673,-6.6626,-14.1257,17.1318,23.6390
